In [1]:

import boto3
import pandas as pd

In [2]:
sm = boto3.Session().client('runtime.sagemaker')


In [3]:
def infer(endpoint, input_data):
    response = sm.invoke_endpoint(EndpointName=endpoint, ContentType='text/csv', Accept='text/csv', Body=input_data)
    prediction = response['Body'].read().decode("utf-8")
    return prediction

In [15]:
endpoint = 'covid-18-150520'
data = '2,12,20,2,20,20,67,2,1,39,1,97,2,2,2,2,2,2,2,2,1,2,2,2,2,99,2,3.0'

prediction = float(infer(endpoint, data))
print('prediction:', prediction)

prediction: 1.0


In [6]:

DATA_PATH = 'data/'
df = pd.read_csv('sagemaker-autopilot/data/covid-test6.csv')

In [12]:
tp = tn = fp = fn = 0
two = one = 0

for index, row in df.iterrows():
    label = float(row['INTUBADO'])
    input_data = ','.join(row.drop('INTUBADO').astype(str).tolist())
    prediction = float(infer(endpoint, input_data))

    if label == 1.0:
        two=two+1
        if prediction == 1.0:
            tp=tp+1
        else:
            fn=fn+1
    else:
        one=one+1
        if prediction == 2.0:
            tn=tn+1
        else:
            fp=fp+1

In [13]:
print ("%d %d" % (tn, fp))
print ("%d %d" % (fn, tp))
print ("one", two, "-- two", one)


accuracy  = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
recall    = tn/(tn+fn)
f1        = (2*precision*recall)/(precision+recall)

print ("%.4f %.4f %.4f %.4f" % (accuracy, precision, recall, f1))

452 143
92 534
two 626 -- One 595
0.8075 0.7888 0.8309 0.8093


Accuracy is used when the True Positives and True negatives are more important while F1-score is used when the False Negatives and False Positives are crucial